<a href="https://colab.research.google.com/github/mihir0804/AI_DevOps_Pipeline_Optimize/blob/main/CI_CD_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install plotly pandas numpy

# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Simulate a dataset
np.random.seed(42)
n = 200

df_simulated = pd.DataFrame({
    'files_changed': np.random.randint(1, 20, n),
    'lines_changed': np.random.randint(10, 1000, n),
    'num_tests': np.random.randint(0, 50, n),
    'test_coverage': np.random.uniform(0.5, 1.0, n),
    'code_complexity': np.random.randint(1, 10, n),
    'build_time_minutes': np.random.uniform(5, 60, n),
    'cpu_usage': np.random.uniform(0.5, 3.0, n),
    'memory_usage_gb': np.random.uniform(1, 16, n),
    'author': np.random.choice(['alice','bob','charlie','diana','eve','frank'], n),
    'deploy_strategy': np.random.choice(['blue-green','canary','rolling'], n),
    'hours_since_last_commit': np.random.randint(1, 72, n),
})

# Define build success probability
df_simulated['predicted_success_prob'] = np.clip(
    0.8 - 0.02*df_simulated['code_complexity'] + 0.01*df_simulated['test_coverage']*100 - 0.001*df_simulated['lines_changed'], 0, 1
)

# Binary outcome
df_simulated['build_success'] = np.random.binomial(1, df_simulated['predicted_success_prob'])

# Suggested strategy
df_simulated['suggested_strategy'] = np.where(df_simulated['predicted_success_prob']>0.7,'rolling','canary')

In [3]:
fig = px.scatter(
    df_simulated,
    x='predicted_success_prob',
    y='build_time_minutes',
    color='build_success',
    hover_data=['author','files_changed','lines_changed','num_tests'],
    title='Build Time vs Predicted Success Probability'
)
fig.show()

In [4]:
fig = px.histogram(
    df_simulated,
    x='predicted_success_prob',
    color='build_success',
    nbins=20,
    title='Distribution of Predicted Success Probability'
)
fig.show()

In [5]:
author_summary = df_simulated.groupby('author')['build_success'].mean().reset_index()
fig = px.bar(author_summary, x='author', y='build_success', title='Build Success Rate per Author')
fig.show()

In [6]:
fig = px.scatter(
    df_simulated,
    x='code_complexity',
    y='build_time_minutes',
    color='build_success',
    hover_data=['author','lines_changed'],
    title='Build Time vs Code Complexity'
)
fig.show()

In [7]:
fig = px.scatter(
    df_simulated,
    x='cpu_usage',
    y='memory_usage_gb',
    color='build_success',
    size='build_time_minutes',
    hover_data=['author','num_tests'],
    title='CPU vs Memory Usage'
)
fig.show()

In [8]:
strategy_summary = df_simulated.groupby('deploy_strategy')['build_success'].mean().reset_index()
fig = px.bar(strategy_summary, x='deploy_strategy', y='build_success', title='Success Rate by Deployment Strategy')
fig.show()

In [9]:
fig = px.scatter(
    df_simulated,
    x='predicted_success_prob',
    y='build_time_minutes',
    color='suggested_strategy',
    hover_data=['author','deploy_strategy'],
    title='Suggested Strategy based on Predicted Success'
)
fig.show()

In [10]:
df_simulated['lines_per_file'] = df_simulated['lines_changed'] / df_simulated['files_changed']
df_simulated['complexity_coverage_ratio'] = df_simulated['code_complexity'] / df_simulated['test_coverage']

fig = px.scatter(
    df_simulated,
    x='lines_per_file',
    y='complexity_coverage_ratio',
    color='build_success',
    hover_data=['author','files_changed'],
    title='Lines per File vs Complexity/Coverage Ratio'
)
fig.show()

In [11]:
# Combine two plots in one figure
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Build Success Rate by Author", "Success Rate by Deployment Strategy"))

# Author success rate
fig.add_trace(
    go.Bar(x=author_summary['author'], y=author_summary['build_success'], name='Author Success Rate'),
    row=1, col=1
)

# Deployment strategy success
fig.add_trace(
    go.Bar(x=strategy_summary['deploy_strategy'], y=strategy_summary['build_success'], name='Strategy Success Rate'),
    row=1, col=2
)

fig.update_layout(title_text="Interactive Build Dashboard", showlegend=False)
fig.show()

In [12]:
import plotly.express as px

# Dropdown filter by author
authors = df_simulated['author'].unique()

fig = px.scatter(
    df_simulated,
    x='predicted_success_prob',
    y='build_time_minutes',
    color='build_success',
    hover_data=['author','files_changed','lines_changed','num_tests'],
    title='Build Time vs Predicted Success Probability'
)

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[dict(label=author,
                          method='update',
                          args=[{'visible':[a==author for a in df_simulated['author']]},
                                {'title':f'Build Time vs Predicted Success Probability: {author}'}])
                    for author in authors] +
                    [dict(label='All',
                          method='update',
                          args=[{'visible':[True]*len(df_simulated)},
                                {'title':'Build Time vs Predicted Success Probability: All Authors'}])]
        )
    ]
)

fig.show()

In [13]:
strategies = df_simulated['deploy_strategy'].unique()

fig = px.bar(
    df_simulated.groupby('deploy_strategy')['build_success'].mean().reset_index(),
    x='deploy_strategy',
    y='build_success',
    title='Success Rate by Deployment Strategy'
)

# Dropdown menu for strategy
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[dict(label=strategy,
                          method='update',
                          args=[{'visible':[s==strategy for s in df_simulated['deploy_strategy']]},
                                {'title':f'Success Rate: {strategy}'}])
                    for strategy in strategies] +
                    [dict(label='All',
                          method='update',
                          args=[{'visible':[True]*len(df_simulated)},
                                {'title':'Success Rate by Deployment Strategy: All'}])]
        )
    ]
)

fig.show()

In [14]:
# Interactive dashboard with multiple dropdowns
from ipywidgets import interact

def filter_dashboard(author='All', strategy='All'):
    df_filtered = df_simulated.copy()
    if author != 'All':
        df_filtered = df_filtered[df_filtered['author']==author]
    if strategy != 'All':
        df_filtered = df_filtered[df_filtered['deploy_strategy']==strategy]

    fig = px.scatter(
        df_filtered,
        x='predicted_success_prob',
        y='build_time_minutes',
        color='build_success',
        hover_data=['author','deploy_strategy','files_changed','lines_changed'],
        title=f'Build Dashboard: Author={author}, Strategy={strategy}'
    )
    fig.show()

interact(filter_dashboard,
         author=['All'] + list(df_simulated['author'].unique()),
         strategy=['All'] + list(df_simulated['deploy_strategy'].unique()))

interactive(children=(Dropdown(description='author', options=('All', 'eve', 'charlie', 'bob', 'frank', 'diana'…

<function __main__.filter_dashboard(author='All', strategy='All')>